In [ ]:
from __future__ import print_function, division, absolute_import

As a preamble - note that JuPyTer notebooks are formatted using [markdown](http://daringfireball.net/projects/markdown/), a text-to-html conversion tool. If you are not familiar, there are [cheatsheets](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) available with all the basic functionality that you will need.

# Building Better Models for Inference:

# How to construct practical models for existing tools

**Version 0.1**

In this notebook, we will walk through fitting a real light curve from a tidal disruption event (TDE), the destruction and accretion of a star by a supermassive black hole, using two different approaches.

As mentioned in the lecture, there are different kinds of models one can apply to a set of data. A code I have written, MOSFiT, is an attempt to provide a framework for building models that can be used within other optimizers/samplers. While MOSFiT can run independently, in the notebook below we will simple be using it as a "black box" function for use in external optimization routines.

Our first approach will be using the `tde` model in MOSFiT. This model uses both interpolation tables and integrations, making an analytical derivative not available. Our second approach will be to construct a simple analytical function to fit the same data. We will then be comparing performance, both in terms of the quality of the resulting solution, but also the speed by which the solution was computed, and in how we relate our solution to what transpired in this event.

* * *

By J Guillochon (Harvard)

*We will be mostly using the mosfit package and scipy routines. Both are available via conda.*

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import mosfit
import time

%matplotlib notebook

## Problem 1) Creating DSFP Problems

We have found that modular notebooks work best. Thus the typical construction includes $\sim3-5$ capital P Problems, each with sub-problems a, b, c, d, etc. The Problems address major ideas from the lecture, while the sub-problems directly address the nitty gritty details related to implementation of the idea. Most sub-problems require some measure of coding (*typically $\lesssim 10$ lines in a single sub-problem is best*), though some simply ask for responses regarding the data or the code that has just been written. 

The modular structure allows the students to move at their own pace, and at this point they all understand that problems get progressively more difficult throughout the notebook. Within the time that is allotted, some students finish and others do not. Following the Problems, there is always a **Challenge Problem** for the students that finish early. These Challenge Problems are always more difficult and typically are not as well structured as the earlier Problems in the notebook.

**An essential tip** - in my experience it is by *far* best to start by writing the solutions to the notebook. This ensures that all the code works and behaves as expected [it also helps control the total time necessary to complete the problems]. Then the notebook for the students can be created by copying the solutions and removing important portions of the code using `# complete` as you will see in the examples below.

It is often, though not always, useful to include code that the students run to load data for the notebook. Here is an example to load some columns from SDSS. As this data is required for the exercise, this is not considered a problem.

In [25]:
# Load the data from the Open Supernova Catalog.
# Note: if loading the data doesn't work, I have a local copy.
my_printer = mosfit.printer.Printer(quiet=True)  # just to avoid spamming from MOSFiT routines.
my_fetcher = mosfit.fetcher.Fetcher(printer=my_printer)

fetched = my_fetcher.fetch('/Users/james/Downloads/mosfit/PS16dtm.json')[0]

my_model = mosfit.model.Model(model='tde', printer=my_printer)
my_model.load_data(my_fetcher.load_data(fetched), event_name=fetched['name'])

x = np.random.rand(100, my_model.get_num_free_parameters())

start_time = time.time()
ln_likes = [my_model.ln_likelihood(xx) for xx in x]
stop_time = time.time()

print('{}s per function call.'.format((stop_time - start_time)/100.0))

0.04788065910339356s per function call.


**Problem 1a**

First, let's visualize the data we have downloaded. MOSFiT loads data in a format conforming to the OAC schema specification, which is a JSON diction where the top level of the dictionary is the event's name. The code snippet below will load a JSON dictionary for the event in question, plot the full time series of photometric data within the `photometry` key below.

*Hint: The photometry is a mixture of different data types, and not every entry has the same set of keys. Optical/UV/IR photometry will always have a `band` key. Use the `.get()` function liberally.*

In [35]:
json_data = my_fetcher.load_data(fetched)

times = []
mags = []
for x in json_data[fetched['name']]['photometry']:
    if x.get('band') != 'g':
        continue
    times.append(x['time'])
    mags.append(x['magnitude'])

plt.plot(times,mags)

<IPython.core.display.Javascript object>

**Problem 1b**

What fraction of the stars in the data set are fainter than $r' = 20 \; \mathrm{mag}$? Store the result in a variable called `Nbright_star`.

In [ ]:
bright = SDSSts[ # complete
Nbright_star = # complete
    
print("There are {:d} stars with r' < 20 the data set.".format( # complete

Sometimes it is useful to add an explanation for the acquired results. Or to add some text setting up the next portion of the problem. Alternatively, you may want the students to think about the results and provide a response for what they found, as follows: *in this case it's good to provide a markdown cell for them to write their answer*

**Problem 1c**

Based on your knowledge of SDSS, do your results above make sense?

*Provide your answer to Problem 1c here*

While plots are useful, and often necessary, it's best to keep them compact if possible. We don't want students struggling with plot syntax (unless the specific topic is visualization). Thus, in these cases, it's best to provide code snippets that are more close to being complete. 

**Problem 1d**

For the bright stars, make a scatter plot of `psdMag_r` vs. `deVMag_r`. 

In [ ]:
plt.scatter(SDSSts[(bright) & (star)][ # complete

plt.xtitle('psfMag_r')
plt.ytitle('deVMag_r')
plt.xlimit( # complete
plt.xlimit( # complete

plt.tight_layout()

## Problem 2

When the first idea is complete, begin a new Problem. The notebook continues with this modular structure until the challenge problem. Problems 2a, 2b, 2c, etc. are eventually to be followed by Problem 3... and so on. 

Text explaining what's going on

** Problem 2a **

Definition of Problem 2a.

In [ ]:
code_snippet1 = # complete
code_snippet2 = # complete
code_snippet3 = # complete
code_snippet4 = # complete

Finally, as noted above, the notebook should end with a Challenge Problem in case any of the students finish the notebook before time is up. This problem should be harder and include fewer prompts.

## Challenge Problem

Complete the following problem, which can be arbitrarily difficult.

In [ ]:
# no code snippets provided here